## Imports

In [1]:
import os
from dotenv import load_dotenv
import logging

logger = logging.getLogger("myapp")

from azure.ai.projects.aio import AIProjectClient
from azure.ai.agents.models import (
    Agent,
    AsyncToolSet,
    AzureAISearchTool,
    FilePurpose,
    FileSearchTool,
    Tool,
)
from azure.ai.projects.models import ConnectionType, ApiKeyCredentials
from azure.identity.aio import DefaultAzureCredential
from azure.core.credentials_async import AsyncTokenCredential

load_dotenv()

# Only show my app logs
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logger = logging.getLogger("myapp")  # Use your desired logger name
handler = logging.StreamHandler()
formatter = logging.Formatter('%(levelname)s %(name)s: %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)
logger.propagate = False



## Create Agent

In [2]:
agent = None

proj_endpoint = os.environ.get("AZURE_EXISTING_AIPROJECT_ENDPOINT")
agent_id = os.environ.get("AZURE_AI_AGENT_ID")

async def create_or_update_agent(agent: Agent | None, ai_client: AIProjectClient, creds: AsyncTokenCredential) -> Agent:
    # tool = await get_available_tool(ai_client, creds)
    toolset = AsyncToolSet()
    # toolset.add(tool)
    
    instructions = "You are a helpful assistant. Use the tools provided to answer questions."
    
    if (agent is None):
        logger.info("Creating new agent with resources")
        agent = await ai_client.agents.create_agent(
            model=os.environ["AZURE_AI_AGENT_DEPLOYMENT_NAME"],
            name=os.environ["AZURE_AI_AGENT_NAME"],
            instructions=instructions,
            toolset=toolset
        )
    else:
        logger.info("Updating existing agent with resources")
        agent.instructions = instructions
        agent = await ai_client.agents.update_agent(
            agent_id=agent.id,
            model=os.environ["AZURE_AI_AGENT_DEPLOYMENT_NAME"],
            name=os.environ["AZURE_AI_AGENT_NAME"],
            instructions=instructions,
            toolset=toolset)
    
    return agent

async def initialize_resources() -> Agent:
    try: 
        async with DefaultAzureCredential(exclude_shared_token_cache_credential=True) as creds:
            async with AIProjectClient(credential=creds, endpoint=proj_endpoint) as ai_client: # type: ignore
                if agent_id is not None: 
                    try:
                        agent = await ai_client.agents.get_agent(agent_id)
                        agent = await create_or_update_agent(agent, ai_client, creds)
                        logger.info(f"Found agent by ID: {agent.id}. Updating agnet")
                        return agent
                    except Exception as e:
                        logger.warning(f"Could not retrieve agent by AZURE_AGENT_ID = \"{agent_id}\". Will create a new agent.")
                
                agent_list = ai_client.agents.list_agents()
                if agent_list:
                    async for agent_object in agent_list:
                        if agent_object.name == os.environ["AZURE_AI_AGENT_NAME"]:
                            logger.info(f"Found existing agent named '{agent_object.name}', ID: {agent_object.id}")
                            os.environ["AZURE_AI_AGENT_ID"] = agent_object.id
                            agent = await ai_client.agents.get_agent(os.environ["AZURE_AI_AGENT_ID"])
                            agent = await create_or_update_agent(agent, ai_client, creds)
                            return agent
                
                # Create a new agent
                agent = await create_or_update_agent(None, ai_client, creds)
                os.environ["AZURE_AI_AGENT_ID"] = agent.id
                logger.info(f"Created agent, agent ID: {agent.id}")

                return agent
    except Exception as e:
        logger.info("Error creating agent: {e}", exc_info=True)
        raise RuntimeError(f"Failed to create the agent: {e}")
    

await initialize_resources()

INFO myapp: Updating existing agent with resources
WARNING myapp: Could not retrieve agent by AZURE_AGENT_ID = "". Will create a new agent.
INFO myapp: Found existing agent named 'my_cool_agent', ID: asst_O5CGrHjmFhHgfhgZAKlasHtB
INFO myapp: Updating existing agent with resources


{'id': 'asst_O5CGrHjmFhHgfhgZAKlasHtB', 'object': 'assistant', 'created_at': 1752051705, 'name': 'my_cool_agent', 'description': None, 'model': 'gpt-4.1-mini', 'instructions': 'You are a helpful assistant. Use the tools provided to answer questions.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}